<a href="https://colab.research.google.com/github/AnkitRajSri/AI_Stock_Trader/blob/master/Stock_Market_Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import libraries

In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader # Used to download and work with stock market information

from tqdm import tqdm_notebook, tqdm # Used to visualize any progress in the project
from collections import deque # Used for experience replay

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
tf.__version__

'2.2.0'

### Building the Stock Trader Deep Q Network

In [3]:
class AIStockTrader():
    def __init__(self, state_size, action_space=3, model_name='AITrader'): # Stay, buy or sell
        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque(maxlen=2000)
        self.inventory = []
        self.model_name = model_name

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995

        self.model = self.modelBuilder()
  
    def modelBuilder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
        return model

    def trade(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
        actions = self.model.predict(state)
        return np.argmax(actions[0])

    def batchTrain(self, batch_size):
        batch = []
        for i in range(len(self.memory)-batch_size+1, len(self.memory)):
            batch.append(self.memory[i])

        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target = self.model.predict(state)
            target[0][action] = reward

            self.model.fit(state, target, epochs=1, verbose=0)
        
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

### Data Preprocessing

#### Defining helper functions

##### Sigmoid

In [4]:
def sigmoid(x):
    return 1 / (1+math.exp(-x))

##### Price format 

In [5]:
def stocksPriceFormat(n):
    if n < 0:
        return '-${0:2f}'.format(abs(n))
    else:
        return '${0:2f}'.format(abs(n))

##### Dataset loader

In [6]:
def datasetLoader(stock_name):
    dataset = data_reader.DataReader(stock_name, data_source='yahoo')
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    close = dataset['Close']

    return close

##### State creator

In [7]:
def stateCreator(data, timestep, window_size):
    starting_id = timestep - window_size + 1

    if starting_id >= 0:
        windowed_data = data[starting_id:timestep+1]
    else:
        windowed_data = -starting_id * [data[0]] + list(data[0:timestep+1])

    state = []
    for i in range(window_size-1):
        state.append(sigmoid(windowed_data[i+1]-windowed_data[i]))

    return np.array([state])

##### Loading the dataset

In [8]:
stock_name = 'AAPL'
data = datasetLoader(stock_name)
data

Date
2015-07-06    126.000000
2015-07-07    125.690002
2015-07-08    122.570000
2015-07-09    120.070000
2015-07-10    123.279999
                 ...    
2020-06-25    364.839996
2020-06-26    353.630005
2020-06-29    361.779999
2020-06-30    364.799988
2020-07-01    364.109985
Name: Close, Length: 1258, dtype: float64

### Training the AI Trader

##### Defining the hyperparameters

In [9]:
WINDOW_SIZE = 10
EPISODES = 1000
BATCH_SIZE = 32
DATA_SAMPLES = len(data) - 1

##### Defining the Trader model

In [10]:
trader = AIStockTrader(state_size=WINDOW_SIZE)

In [11]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


##### Training loop

In [ ]:
for episode in range(EPISODES+1):
    print('\nEpisode: {}/{}'.format(episode, EPISODES))
    state = stateCreator(data, 0, WINDOW_SIZE+1)
    
    total_profit = 0
    trader.inventory = []

    for t in tqdm(range(DATA_SAMPLES)):
        action = trader.trade(state)
        next_state = stateCreator(data, t+1, WINDOW_SIZE+1)
        reward = 0

        if action == 1: # Buying
            trader.inventory.append(data[t])
            print('\nAI Trader bought: ', stocksPriceFormat(data[t]))

        elif action == 2 and len(trader.inventory) > 0: # Selling
            buy_price = trader.inventory.pop(0)
            reward = max(data[t]-buy_price, 0)
            total_profit += data[t] - buy_price
            print('\nAI Trader sold: ', stocksPriceFormat(data[t]), " Profit: " + stocksPriceFormat(data[t] - buy_price) )

        if t == DATA_SAMPLES - 1:
            done = True
        else:
            done = False

        trader.memory.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            print('***************************************************')
            print('Total Profit: {}'.format(total_profit))
            print('***************************************************')

        if len(trader.memory) > BATCH_SIZE:
            trader.batchTrain(BATCH_SIZE)

        if episode%10 == 0:
            trader.model.save('ai_trader_{}.h5'.format(episode))

  1%|          | 11/1257 [00:00<00:11, 108.06it/s]


Episode: 0/1000

AI Trader bought:  $122.570000

AI Trader bought:  $120.070000

AI Trader sold:  $123.279999  Profit: $0.709999

AI Trader sold:  $125.660004  Profit: $5.590004

AI Trader bought:  $132.070007

AI Trader sold:  $130.750000  Profit: -$1.320007

AI Trader bought:  $125.160004

AI Trader bought:  $124.500000

AI Trader sold:  $123.379997  Profit: -$1.780006

AI Trader sold:  $122.989998  Profit: -$1.510002

AI Trader bought:  $122.370003

AI Trader bought:  $121.300003


  2%|▏         | 22/1257 [00:00<00:11, 106.39it/s]


AI Trader sold:  $115.400002  Profit: -$6.970001

AI Trader sold:  $115.129997  Profit: -$6.170006

AI Trader bought:  $117.160004

AI Trader bought:  $116.500000

AI Trader sold:  $115.010002  Profit: -$2.150002


  3%|▎         | 33/1257 [00:04<02:31,  8.10it/s] 

...

 43%|████▎     | 536/1257 [18:16<23:47,  1.98s/it]


AI Trader bought:  $157.500000


 43%|████▎     | 537/1257 [18:18<24:08,  2.01s/it]


AI Trader bought:  $157.210007


 43%|████▎     | 538/1257 [18:20<24:19,  2.03s/it]